In [1]:
import sys, pdb, warnings, scipy, matplotlib, sklearn, itertools

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cPickle as pkl
import seaborn as sns

from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFECV

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
sys.setrecursionlimit(15000)
pd.set_option('display.max_columns', 300)
%matplotlib inline

print('scipy: {}'.format(scipy.__version__))
print('numpy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(matplotlib.__version__)) 
print('sklearn: {}'.format(sklearn.__version__))

#our modules see: CS_Project/cspython directory
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped
import cspython.analysis as a
from cspython.merging_processing import combine_dfs

scipy: 0.19.1
numpy: 1.12.1
pandas: 0.20.3
matplotlib: 2.0.2
sklearn: 0.19.0


In [ ]:

with open('../cspython/esl_teams.pkl', 'rb') as f: 
     d = pkl.load(f)
        
big_data = process_scrapped(d)
overview, big_data = big_data

data = combine_dfs(overview, big_data)
data.sort_values('date', inplace=True)
data.index = range(len(data))


In [340]:
#data.to_pickle("JohnTesting.pkl")

In [2]:
data = pd.read_pickle("JohnTesting.pkl")

In [ ]:
data_team = data.loc[(data.loc[:,'player_team_name'] == b) & (data.loc[:,'map'] == a), ].sort_values(by='date',ascending=True)
grouping = data_team.groupby(['player_team_name','match_id'])['ADR'].max()
grouping = pd.DataFrame(grouping)
grouping = grouping.reset_index()
grouping.loc[:,'ADR'] = grouping.loc[:,'ADR'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
grouping = grouping.rename(index=str, columns={'ADR': 'ADR_hist_on_map'})
new_group = pd.concat([new_group, grouping])

In [4]:
def round_win_rate_map_his(data): #gets the historic winning average on each round of each map
    round_num = list(data.round_num.unique())
    team = list(data.player_team_name.unique())
    maps = list(data.map.unique())
    fgrouping = pd.DataFrame()
    for a in team:
        data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by=['date', 'series_id','match_num'],ascending=True)
        grouping = data_team.groupby(['player_team_name','date','series_id','match_num','match_id','map','round_num'])['winner'].max()
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        grouping.winner = grouping.winner.replace(to_replace = a, value = 1)
        for b in maps:
            for c in round_num:
                grouping.loc[:,b+"_round_"+str(c)] = np.nan
                grouping.loc[(grouping.loc[:,"map"]== b)&(grouping.loc[:,'round_num']== c) & (grouping.loc[:,"winner"] == 1),b+"_round_"+str(c)] = 1
                grouping.loc[(grouping.loc[:,"map"]== b)&(grouping.loc[:,'round_num']== c) & (grouping.loc[:,"winner"] != 1),b+"_round_"+str(c)] = 0
                grouping.loc[(grouping.loc[:,"map"]== b)&(grouping.loc[:,'round_num']== c),b+"_round_"+str(c)] = grouping.loc[(grouping.loc[:,"map"]== b)&(grouping.loc[:,'round_num']== c),b+"_round_"+str(c)].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
        grouping = grouping.groupby(['player_team_name','date','series_id','match_num','match_id']).max()
        grouping = grouping.reset_index()
        grouping = grouping.drop(['round_num','winner'], axis = 1, errors = 'ignore')
        grouping = grouping.fillna(method='ffill')

        fgrouping = pd.concat([fgrouping, grouping], axis = 0)

    data = pd.merge(data, fgrouping, on= ['player_team_name','date','series_id','match_num','match_id','map'])
    return data

In [5]:
def round_win_rate_his(data):# gets the historic winning average on each round of all matches
    round_num = list(data.round_num.unique())
    team = list(data.player_team_name.unique())
    maps = list(data.map.unique())
    fgrouping = pd.DataFrame()
    for a in team:
        data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by=['date', 'series_id','match_num'],ascending=True)
        grouping = data_team.groupby(['player_team_name','date','series_id','match_num','match_id','round_num'])['winner'].max()
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        grouping.winner = grouping.winner.replace(to_replace = a, value = 1)
        for c in round_num:
                grouping.loc[:,"round_"+str(c)] = np.nan
                grouping.loc[(grouping.loc[:,'round_num']== c) & (grouping.loc[:,"winner"] == 1),"round_"+str(c)] = 1
                grouping.loc[(grouping.loc[:,'round_num']== c) & (grouping.loc[:,"winner"] != 1),"round_"+str(c)] = 0
                grouping.loc[(grouping.loc[:,'round_num']== c),"round_"+str(c)] = grouping.loc[(grouping.loc[:,'round_num']== c),"round_"+str(c)].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
        grouping = grouping.groupby(['player_team_name','date','series_id','match_num','match_id']).max()
        grouping = grouping.reset_index()
        grouping = grouping.drop(['round_num','winner'], axis = 1, errors = 'ignore')
        grouping = grouping.fillna(method = 'ffill')

        fgrouping = pd.concat([fgrouping, grouping], axis = 0)

    data = pd.merge(data, fgrouping, on= ['player_team_name','date','series_id','match_num','match_id'])
    return data

    


In [6]:
data = round_win_rate_his(data)

In [ ]:
def win_previous_match(data):
    team = list(data.player_team_name.unique())
    maps = list(data.map.unique())
    fgrouping = pd.DataFrame()
    for a in team:
        data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by=['date', 'series_id','match_num'],ascending=True)
        grouping = data_team.groupby(['player_team_name','date','series_id','match_num','match_id','round_num'])['winner_of_match'].max()
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        
    
    

In [ ]:
def win_previous_match_map(data)

In [45]:
data.sort_values(by=['date', 'match_num'],ascending=True)

,index,map,round_num,half,match_id,series_id,ending,CT,T,side_winner,winner,team_A,team_B,team_A_score,team_B_score,match_num,team_players,K-D,+/-,ADR,KAST,Rating2.0,nicknames,jks_who_kill_who,Stewie2K_who_kill_who,NAF_awp_kills,player_team_name,Stewie2K_awp_kills,NAF_who_kill_who,NAF_first_kills,Nifty_first_kills,Nifty_who_kill_who,AZR_first_kills,RUSH_first_kills,RUSH_awp_kills,tarik_awp_kills,AZR_awp_kills,Skadoodle_first_kills,USTILO_awp_kills,Stewie2K_first_kills,tarik_who_kill_who,AZR_who_kill_who,tarik_first_kills,Skadoodle_who_kill_who,USTILO_first_kills,winner_of_match,autimatic_who_kill_who,date,autimatic_awp_kills,Skadoodle_awp_kills,autimatic_first_kills,USTILO_who_kill_who,loser_of_match,jks_first_kills,team_B_name,Nifty_awp_kills,jks_awp_kills,RUSH_who_kill_who,team_A_name,mixwell_who_kill_who,HS_awp_kills,friberg_awp_kills,allu_who_kill_who,mixwell_awp_kills,HS_first_kills,Magisk_awp_kills,Magisk_who_kill_who,Magisk_first_kills,mixwell_first_kills,allu_first_kills,allu_awp_kills,HS_who_kill_who,friberg_who_kill_who,friberg_first_kills,chelo_first_kills,chelo_who_kill_who,nitr0_who_kill_who,nitr0_awp_kills,SHOOWTiME_who_kill_who,NEKIZ_first_kills,EliGE_first_kills,jdm64_awp_kills,NEKIZ_awp_kills,Twistzz_who_kill_who,stanislaw_who_kill_who,PKL_who_kill_who,Twistzz_awp_kills,yeL_first_kills,PKL_awp_kills,yeL_awp_kills,PKL_first_kills,SHOOWTiME_awp_kills,yeL_who_kill_who,stanislaw_first_kills,EliGE_awp_kills,chelo_awp_kills,Twistzz_first_kills,EliGE_who_kill_who,stanislaw_awp_kills,SHOOWTiME_first_kills,jdm64_who_kill_who,jdm64_first_kills,NEKIZ_who_kill_who,nitr0_first_kills,RIKO_who_kill_who,Dallas_who_kill_who,CONNOR93_who_kill_who,ryx_who_kill_who,Brehze_awp_kills,Wardell_first_kills,AnJ_first_kills,Dallas_awp_kills,ryx_first_kills,AnJ_who_kill_who,RIKO_first_kills,CONNOR93_first_kills,AnJ_awp_kills,Wardell_awp_kills,CeRq_first_kills,FugLy_who_kill_who,Brehze_first_kills,seb_who_kill_who,ryx_awp_kills,CONNOR93_awp_kills,RIKO_awp_kills,Brehze_who_kill_who,seb_awp_kills,CeRq_who_kill_who,CeRq_awp_kills,Dallas_first_kills,seb_first_kills,Wardell_who_kill_who,FugLy_awp_kills,FugLy_first_kills,daps_awp_kills,daps_who_kill_who,daps_first_kills,destiny_first_kills,FNS_awp_kills,shz_first_kills,Rickeh_awp_kills,koosta_who_kill_who,FNS_first_kills,destiny_awp_kills,reltuC_awp_kills,reltuC_who_kill_who,shz_who_kill_who,FNS_who_kill_who,Rickeh_who_kill_who,reltuC_first_kills,...,ShahZaM_awp_kills,seang@res_first_kills,SicK_awp_kills,devoduvek_first_kills,AmaNEk_first_kills,SicK_first_kills,ShahZaM_who_kill_who,ShahZaM_first_kills,AmaNEk_who_kill_who,Warden_awp_kills,yay_awp_kills,ANDROID-X23_who_kill_who,Subroza_awp_kills,Subroza_who_kill_who,dephh_awp_kills,ANDROID-X23_first_kills,stan1ey_first_kills,yay_who_kill_who,yay_first_kills,stan1ey_who_kill_who,ANDROID-X23_awp_kills,Warden_first_kills,Subroza_first_kills,dephh_first_kills,stan1ey_awp_kills,dephh_who_kill_who,Warden_who_kill_who,shroud_who_kill_who,n0thing_first_kills,shroud_awp_kills,n0thing_who_kill_who,n0thing_awp_kills,shroud_first_kills,horvy_awp_kills,zakk_first_kills,zakk_awp_kills,horvy_who_kill_who,horvy_first_kills,zakk_who_kill_who,shinobi_first_kills,Hiko_awp_kills,Grt_first_kills,Grt_who_kill_who,Grt_awp_kills,vice_first_kills,vice_who_kill_who,vice_awp_kills,shinobi_who_kill_who,Hiko_who_kill_who,Hiko_first_kills,witmer_first_kills,shinobi_awp_kills,witmer_awp_kills,witmer_who_kill_who,Slemmy_first_kills,Surreal_who_kill_who,Slemmy_awp_kills,Surreal_awp_kills,arya_who_kill_who,arya_awp_kills,desi_who_kill_who,desi_first_kills,Surreal_first_kills,arya_first_kills,Slemmy_who_kill_who,desi_awp_kills,zqkS_first_kills,zqkS_awp_kills,zqkS_who_kill_who,wrath_awp_kills,cadiaN_who_kill_who,cadiaN_awp_kills,wrath_first_kills,cadiaN_first_kills,wrath_who_kill_who,Jayzwalkingz_who_kill_who,Jayzwalkingz_awp_kills,Jayzwalkingz_first_kills,Uber_who_kill_who,Uber_first_kills,Uber_awp_kills,hazed_awp_kills,hazed_first_kills,hazed_who_kill_who,neptune_who_kill_who,nep

In [25]:
data.loc[(data.loc[:,'player_team_name'] == 'Splyce') & (data.loc[:,'round_num'] == 1) ,'round_18']

116       0.500000
135       0.500000
157       0.500000
190       0.500000
221       0.500000
388       0.000000
409       0.000000
415       0.000000
437       0.000000
477       0.000000
22374     0.250000
22430     0.250000
22434     0.250000
22448     0.250000
22473     0.250000
22486     0.400000
22557     0.400000
22561     0.400000
22592     0.400000
22606     0.400000
22621     0.333333
22636     0.333333
22653     0.333333
22683     0.333333
22703     0.333333
22970     0.333333
22999     0.333333
23012     0.333333
23031     0.333333
23034     0.333333
            ...   
148362    0.478261
148377    0.478261
148409    0.478261
148428    0.478261
148446    0.478261
151041    0.468085
151058    0.468085
151110    0.468085
151121    0.468085
151160    0.468085
151391    0.458333
151407    0.458333
151427    0.458333
151489    0.458333
151505    0.458333
151516    0.469388
151542    0.469388
151574    0.469388
151583    0.469388
151594    0.469388
153049    0.450980
153050    0.

In [ ]:
grouping.loc[(grouping.loc[:,"map"]== b),] = grouping.loc[(grouping.loc[:,"map"]== b),].expanding(
    
    grouping.columns.difference(['player_team_name','date','match_id','map']

In [331]:
data

,index,map,round_num,half,match_id,series_id,ending,CT,T,side_winner,winner,team_A,team_B,team_A_score,team_B_score,match_num,team_players,K-D,+/-,ADR,KAST,Rating2.0,nicknames,friberg_awp_kills,player_team_name,fer_awp_kills,allu_first_kills,HS_awp_kills,allu_who_kill_who,FalleN_who_kill_who,HS_first_kills,felps_awp_kills,FalleN_first_kills,TACO_who_kill_who,fer_first_kills,coldzera_who_kill_who,mixwell_who_kill_who,friberg_first_kills,mixwell_awp_kills,coldzera_awp_kills,Magisk_who_kill_who,felps_first_kills,winner_of_match,Magisk_awp_kills,date,TACO_first_kills,friberg_who_kill_who,fer_who_kill_who,felps_who_kill_who,loser_of_match,HS_who_kill_who,Magisk_first_kills,team_B_name,coldzera_first_kills,mixwell_first_kills,FalleN_awp_kills,allu_awp_kills,team_A_name,TACO_awp_kills,Skadoodle_awp_kills,tarik_first_kills,Stewie2K_who_kill_who,RUSH_first_kills,RUSH_awp_kills,Skadoodle_who_kill_who,tarik_awp_kills,Stewie2K_awp_kills,autimatic_who_kill_who,Skadoodle_first_kills,RUSH_who_kill_who,Stewie2K_first_kills,autimatic_awp_kills,tarik_who_kill_who,autimatic_first_kills,FNS_awp_kills,SEMPHIS_first_kills,Drone_who_kill_who,roca_awp_kills,SEMPHIS_who_kill_who,FNS_who_kill_who,FNS_first_kills,Drone_awp_kills,DAVEY_awp_kills,nahtE_first_kills,koosta_first_kills,roca_first_kills,DAVEY_who_kill_who,m1tch_awp_kills,reltuC_first_kills,Drone_first_kills,Rickeh_who_kill_who,nahtE_who_kill_who,m1tch_first_kills,koosta_who_kill_who,reltuC_who_kill_who,nahtE_awp_kills,reltuC_awp_kills,SEMPHIS_awp_kills,Rickeh_first_kills,koosta_awp_kills,roca_who_kill_who,Rickeh_awp_kills,DAVEY_first_kills,m1tch_who_kill_who,AnJ_who_kill_who,Brehze_first_kills,daps_who_kill_who,AnJ_awp_kills,daps_first_kills,FugLy_who_kill_who,daps_awp_kills,Brehze_who_kill_who,Brehze_awp_kills,AnJ_first_kills,FugLy_awp_kills,CeRq_first_kills,CeRq_who_kill_who,CeRq_awp_kills,FugLy_first_kills,seang@res_who_kill_who,SicK_who_kill_who,seang@res_awp_kills,SicK_awp_kills,devoduvek_first_kills,Slemmy_awp_kills,AmaNEk_first_kills,dephh_first_kills,ShahZaM_first_kills,AmaNEk_awp_kills,yay_awp_kills,ANDROID-X23_first_kills,ptr_who_kill_who,yay_first_kills,seang@res_first_kills,SicK_first_kills,dephh_awp_kills,ShahZaM_awp_kills,Slemmy_first_kills,ANDROID-X23_who_kill_who,devoduvek_awp_kills,Slemmy_who_kill_who,ANDROID-X23_awp_kills,ShahZaM_who_kill_who,devoduvek_who_kill_who,ptr_awp_kills,ptr_first_kills,yay_who_kill_who,dephh_who_kill_who,AmaNEk_who_kill_who,chelo_first_kills,...,Overpass_round_14,Overpass_round_11,Overpass_round_10,Overpass_round_9,Overpass_round_8,Overpass_round_7,Overpass_round_6,Overpass_round_5,Overpass_round_4,Overpass_round_3,Overpass_round_2,Overpass_round_1,Overpass_round_21,Overpass_round_12,Overpass_round_20,Overpass_round_18,Overpass_round_17,Overpass_round_16,Overpass_round_15,Overpass_round_13,Overpass_round_19,Overpass_round_28,Overpass_round_27,Overpass_round_26,Overpass_round_25,Overpass_round_24,Overpass_round_23,Overpass_round_22,Overpass_round_29,Overpass_round_30,Cobblestone_round_14,Cobblestone_round_11,Cobblestone_round_10,Cobblestone_round_9,Cobblestone_round_8,Cobblestone_round_7,Cobblestone_round_6,Cobblestone_round_5,Cobblestone_round_4,Cobblestone_round_3,Cobblestone_round_2,Cobblestone_round_1,Cobblestone_round_21,Cobblestone_round_12,Cobblestone_round_20,Cobblestone_round_18,Cobblestone_round_17,Cobblestone_round_16,Cobblestone_round_15,Cobblestone_round_13,Cobblestone_round_19,Cobblestone_round_28,Cobblestone_round_27,Cobblestone_round_26,Cobblestone_round_25,Cobblestone_round_24,Cobblestone_round_23,Cobblestone_round_22,Cobblestone_round_29,Cobblestone_round_30,Inferno_round_14,Inferno_round_11,Inferno_round_10,Inferno_round_9,Inferno_round_8,Inferno_round_7,Inferno_round_6,Inferno_round_5,Inferno_round_4,Inferno_round_3,Inferno_round_2,Inferno_round_1,Inferno_round_21,Inferno_round_12,Inferno_round_20,Inferno_round_18,Inferno_round_17,Inferno_round_16,Inferno_round_15,Inferno_round_13,Inferno_round_19,Inferno_round_28,Inferno_round_27

In [367]:
data.loc[(data.loc[:,'player_team_name'] == 'Splyce') & (data.loc[:,'map']== 'Nuke') &(data.loc[:,'round_num'] == 1)].sort_values(by=['date', 'match_num'],ascending=True)

,index,map,round_num,half,match_id,series_id,ending,CT,T,side_winner,winner,team_A,team_B,team_A_score,team_B_score,match_num,team_players,K-D,+/-,ADR,KAST,Rating2.0,nicknames,jks_who_kill_who,Stewie2K_who_kill_who,NAF_awp_kills,player_team_name,Stewie2K_awp_kills,NAF_who_kill_who,NAF_first_kills,Nifty_first_kills,Nifty_who_kill_who,AZR_first_kills,RUSH_first_kills,RUSH_awp_kills,tarik_awp_kills,AZR_awp_kills,Skadoodle_first_kills,USTILO_awp_kills,Stewie2K_first_kills,tarik_who_kill_who,AZR_who_kill_who,tarik_first_kills,Skadoodle_who_kill_who,USTILO_first_kills,winner_of_match,autimatic_who_kill_who,date,autimatic_awp_kills,Skadoodle_awp_kills,autimatic_first_kills,USTILO_who_kill_who,loser_of_match,jks_first_kills,team_B_name,Nifty_awp_kills,jks_awp_kills,RUSH_who_kill_who,team_A_name,mixwell_who_kill_who,HS_awp_kills,friberg_awp_kills,allu_who_kill_who,mixwell_awp_kills,HS_first_kills,Magisk_awp_kills,Magisk_who_kill_who,Magisk_first_kills,mixwell_first_kills,allu_first_kills,allu_awp_kills,HS_who_kill_who,friberg_who_kill_who,friberg_first_kills,chelo_first_kills,chelo_who_kill_who,nitr0_who_kill_who,nitr0_awp_kills,SHOOWTiME_who_kill_who,NEKIZ_first_kills,EliGE_first_kills,jdm64_awp_kills,NEKIZ_awp_kills,Twistzz_who_kill_who,stanislaw_who_kill_who,PKL_who_kill_who,Twistzz_awp_kills,yeL_first_kills,PKL_awp_kills,yeL_awp_kills,PKL_first_kills,SHOOWTiME_awp_kills,yeL_who_kill_who,stanislaw_first_kills,EliGE_awp_kills,chelo_awp_kills,Twistzz_first_kills,EliGE_who_kill_who,stanislaw_awp_kills,SHOOWTiME_first_kills,jdm64_who_kill_who,jdm64_first_kills,NEKIZ_who_kill_who,nitr0_first_kills,RIKO_who_kill_who,Dallas_who_kill_who,CONNOR93_who_kill_who,ryx_who_kill_who,Brehze_awp_kills,Wardell_first_kills,AnJ_first_kills,Dallas_awp_kills,ryx_first_kills,AnJ_who_kill_who,RIKO_first_kills,CONNOR93_first_kills,AnJ_awp_kills,Wardell_awp_kills,CeRq_first_kills,FugLy_who_kill_who,Brehze_first_kills,seb_who_kill_who,ryx_awp_kills,CONNOR93_awp_kills,RIKO_awp_kills,Brehze_who_kill_who,seb_awp_kills,CeRq_who_kill_who,CeRq_awp_kills,Dallas_first_kills,seb_first_kills,Wardell_who_kill_who,FugLy_awp_kills,FugLy_first_kills,daps_awp_kills,daps_who_kill_who,daps_first_kills,destiny_first_kills,FNS_awp_kills,shz_first_kills,Rickeh_awp_kills,koosta_who_kill_who,FNS_first_kills,destiny_awp_kills,reltuC_awp_kills,reltuC_who_kill_who,shz_who_kill_who,FNS_who_kill_who,Rickeh_who_kill_who,reltuC_first_kills,...,Overpass_round_8,Overpass_round_18,Overpass_round_19,Overpass_round_20,Overpass_round_21,Overpass_round_22,Overpass_round_23,Overpass_round_24,Overpass_round_25,Overpass_round_26,Overpass_round_27,Overpass_round_28,Overpass_round_1,Overpass_round_17,Overpass_round_2,Overpass_round_4,Overpass_round_5,Overpass_round_6,Overpass_round_7,Overpass_round_9,Overpass_round_10,Overpass_round_11,Overpass_round_12,Overpass_round_13,Overpass_round_14,Overpass_round_15,Overpass_round_3,Overpass_round_16,Overpass_round_29,Overpass_round_30,Cobblestone_round_8,Cobblestone_round_18,Cobblestone_round_19,Cobblestone_round_20,Cobblestone_round_21,Cobblestone_round_22,Cobblestone_round_23,Cobblestone_round_24,Cobblestone_round_25,Cobblestone_round_26,Cobblestone_round_27,Cobblestone_round_28,Cobblestone_round_1,Cobblestone_round_17,Cobblestone_round_2,Cobblestone_round_4,Cobblestone_round_5,Cobblestone_round_6,Cobblestone_round_7,Cobblestone_round_9,Cobblestone_round_10,Cobblestone_round_11,Cobblestone_round_12,Cobblestone_round_13,Cobblestone_round_14,Cobblestone_round_15,Cobblestone_round_3,Cobblestone_round_16,Cobblestone_round_29,Cobblestone_round_30,Inferno_round_8,Inferno_round_18,Inferno_round_19,Inferno_round_20,Inferno_round_21,Inferno_round_22,Inferno_round_23,Inferno_round_24,Inferno_round_25,Inferno_round_26,Inferno_round_27,Inferno_round_28,Inferno_round_1,Inferno_round_17,Inferno_round_2,Inferno_round_4,Inferno_round_5,Inferno_round_6,Inferno_round_7,Inferno_round_9,Inferno_round_10,Inferno_round_11,Inferno_round_12,Inferno_round_13,Inferno_round_14,Infer

In [6]:
def create_opponent_team_col(data):
    data.loc[:,'player_team_opponent'] = np.nan
    data.loc[(data['team_A_name'] != data['player_team_name']),'player_team_opponent'] = data.loc[:,'team_A_name']
    data.loc[(data['team_B_name'] != data['player_team_name']),'player_team_opponent'] = data.loc[:,'team_B_name']
    return data

def create_fwa_dr_columns(data, col_list):  # first k , awp, who, divided by rounds 
    columns = pd.Series(data.columns)
    for a in col_list:
        col = columns[columns.str.contains(a)]
        data[a+'_sum_dr'] = data[col].convert_objects(convert_numeric = True).sum(axis = 1) / (data['team_A_score'] + data['team_B_score'])*100
    data.loc[:, data.columns != 'date'] = data.loc[:, data.columns != 'date'].apply(pd.to_numeric, errors='ignore')
    return data

def create_player_columns(data):
    names = data.nicknames.unique()
    for a in names:
        data.loc[:,a] = 0
        data.loc[data.loc[:,'nicknames'] == a, a] = 5   # its 5 so that when you group by team it becomes 1 
    return data
    
def apply_nummeric_and_group_as_match(data):
    r = data.loc[:, data.columns != 'date'].apply(pd.to_numeric, errors='ignore')
    r['date'] = data.date
    data = r
    data = data.fillna(0)
    data_match = data.groupby(['match_id', 'player_team_name', 'date', 'team_A_name', 'team_B_name', 'series_id', 'map', 'winner_of_match', 'loser_of_match','player_team_opponent']).mean()
    data_match = pd.DataFrame(data_match)
    data_match = data_match.reset_index()
    return data_match

def player_based_column_making(data): # these are the functions that are non historic and arent grouped by match_id
    col_list = ['first_kills', 'who_kill_who', 'awp_kills']
    data = create_opponent_team_col(data)
    data = create_fwa_dr_columns(data, col_list)
    data = create_player_columns(data)
    return data

def create_historic_data(data):
    col_list = ['first_kills', 'who_kill_who','awp_kills']
    data = create_fwadr_his(data, col_list)
    data = create_matches_count(data)
    data = create_avdamage_his(data)
    data = create_avdamage_map_his(data)
    data = create_faw_map_his(data, col_list)
    return data

def create_matches_count(data): # how many matches a team has played
    teams = list(data.player_team_name.unique())
    new_group = pd.DataFrame()
    for a in teams:   
        data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by='date',ascending=True)
        grouping = data_team.groupby(['player_team_name','date','match_id'])['ADR'].count()
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        grouping.loc[:,'ADR'] = grouping.loc[:,'ADR'].expanding(min_periods=1, freq=None, center=False, axis=0).sum()
        grouping = grouping.rename(index=str, columns={'ADR': 'matches_played_team'})
        new_group = pd.concat([new_group, grouping])
    data = pd.merge(data, new_group, on = ['player_team_name','match_id','date'])

    return data

def create_avdamage_his(data):  # column with historic average damage of individual !!!
    teams = list(data.player_team_name.unique())
    new_group = pd.DataFrame()
    for a in teams:   
        data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by='date',ascending=True)
        grouping = data_team.groupby(['player_team_name','date','match_id'])['ADR'].max()
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        grouping.loc[:,'ADR'] = grouping.loc[:,'ADR'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
        grouping = grouping.rename(index=str, columns={'ADR': 'ADR_hist'})
        new_group = pd.concat([new_group, grouping])
    data = pd.merge(data, new_group, on = ['player_team_name','match_id','date'])

    return data

def create_avdamage_map_his(data):# historic average damage of individual for each map !!!
    teams = list(data.player_team_name.unique())
    maps = list(data.map.unique())
    new_group = pd.DataFrame()
    for a in maps:
        for b in teams:   
            data_team = data.loc[(data.loc[:,'player_team_name'] == b) & (data.loc[:,'map'] == a), ].sort_values(by='date',ascending=True)
            grouping = data_team.groupby(['player_team_name','match_id'])['ADR'].max()
            grouping = pd.DataFrame(grouping)
            grouping = grouping.reset_index()
            grouping.loc[:,'ADR'] = grouping.loc[:,'ADR'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
            grouping = grouping.rename(index=str, columns={'ADR': 'ADR_hist_on_map'})
            new_group = pd.concat([new_group, grouping])
    
    data = pd.merge(data, new_group, on = ['match_id','player_team_name'])

    return data

def create_fwadr_his(data, col_list):
    teams = list(data.player_team_name.unique())
    for b in col_list:
        new_group = pd.DataFrame()
        for a in teams:   
            data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by='date',ascending=True)
            grouping = data_team.groupby(['player_team_name','date','match_id'])[b+'_sum_dr'].max()
            grouping = pd.DataFrame(grouping)
            grouping = grouping.reset_index()
            grouping.loc[:,b +'_sum_dr'] = grouping.loc[:,b +'_sum_dr'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
            grouping = grouping.rename(index=str, columns={b +'_sum_dr': b + '_sum_dr_hist'})
            new_group = pd.concat([new_group, grouping])
        data = pd.merge(data, new_group, on = ['player_team_name','match_id','date'])
    return data

def create_faw_map_his(data, col_list):
    teams = list(data.player_team_name.unique())
    maps = list(data.map.unique())
    for b in col_list:
        new_group = pd.DataFrame()
        for i in maps:
            for a in teams:   
                data_team = data.loc[(data.loc[:,'player_team_name'] == a) & (data.loc[:,'map'] == i), ].sort_values(by='date',ascending=True)
                grouping = data_team.groupby(['player_team_name','match_id'])[b+'_sum_dr'].max()
                grouping = pd.DataFrame(grouping)
                grouping = grouping.reset_index()
                grouping.loc[:,b +'_sum_dr'] = grouping.loc[:,b +'_sum_dr'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
                grouping = grouping.rename(index=str, columns={b +'_sum_dr': b + '_sum_dr_hist_on_map'})
                new_group = pd.concat([new_group, grouping])
                
        data = pd.merge(data, new_group, on = ['match_id','player_team_name'])
    return data   

def match_dataset_creation(data):  #creates player based columns, then groups to allow for historic match based columns
    data = player_based_column_making(data)
    data = apply_nummeric_and_group_as_match(data)
    return data

In [13]:
"""
data = match_dataset_creation(data)
data.to_pickle("match_dataset.pkl")
"""

data = pd.read_pickle("aggregated_dataset.pkl")
print data.shape


(1208, 703)


In [7]:
def count_map_win_loss_total(df): #this function assumes the df contains only rows from one team
    team_name = df.player_team_name.iloc[0]
    maps = df.map.unique()
    
    for map_name in maps:
        df[map_name + "_win_his"] = np.nan
        df[map_name + "_loss_his"] = np.nan
        df[map_name + "_total_played"] = np.nan
        
    grouped = df.groupby('map')
    for map_name, map_df in grouped:
        played = pd.Series(range(1,len(map_df)+1), index=map_df.index)
        won = (map_df.winner_of_match == team_name).expanding(1).sum()
        lost = played - won
        
        df.loc[map_df.index, map_name + '_total_played'] = played
        df.loc[map_df.index, map_name + '_win_his'] = won
        df.loc[map_df.index, map_name + '_loss_his'] = lost
        
        df.loc[:, map_name + '_total_played'].fillna(method='ffill', inplace=True)
        df.loc[:, map_name + '_win_his'].fillna(method='ffill', inplace=True)
        df.loc[:, map_name + '_loss_his'].fillna(method='ffill', inplace=True)
        
        df.loc[:, map_name + '_total_played'].fillna(0, inplace=True)
        df.loc[:, map_name + '_win_his'].fillna(0, inplace=True)
        df.loc[:, map_name + '_loss_his'].fillna(0, inplace=True)
        
    return df


def create_map_win_loss_and_per_his_columns(data):  # team total win and loses on map with total times played on map !4!
    data.sort_values('date', inplace=True)
    data.index = range(len(data))
    original_col_order = data.columns.tolist()
    data = data.groupby('player_team_name').apply(count_map_win_loss_total)
    new_cols = data.columns[~data.columns.isin(original_col_order)].tolist()
    data = data.loc[:, original_col_order + new_cols]
    for map_name in data.map.unique():
        data.loc[:, map_name + '_win_perc_map'] = data.loc[:, map_name + '_win_his']/data.loc[:, map_name + '_total_played']
    return data


In [8]:
#  FUNCTIONS BELOW HERE NEED TO BE WORKED ON

In [9]:
#Rounds won and lost in current match
def create_rounds_won_rounds_loss_columns(data):
    team_A_rounds = data.loc[data.player_team_name == data.team_A_name, 'team_A_score']
    team_B_rounds = data.loc[data.player_team_name == data.team_B_name, 'team_B_score']
    rounds_won = pd.concat([team_A_rounds, team_B_rounds])
    data.loc[:, 'rounds_won'] = rounds_won
    
    team_A_rounds = data.loc[data.player_team_name == data.team_B_name, 'team_A_score']
    team_B_rounds = data.loc[data.player_team_name == data.team_A_name, 'team_B_score']
    rounds_lost = pd.concat([team_A_rounds, team_B_rounds])
    data.loc[:, 'rounds_lost'] = rounds_lost
    
    return data

#running total of rounds won/lost vs opponent

def count_rounds_won_vs_opponent(df): #parsing a df of 1 team
    grouped = df.groupby('player_team_opponent')
    for opponent, opponent_df in grouped:
        won_his = opponent_df.loc[:, 'rounds_won'].expanding(1).sum()
        loss_his = opponent_df.loc[:,'rounds_lost'].expanding(1).sum()
        
        df.loc[opponent_df.index, 'rounds_won_vs_'+opponent] = won_his
        df.loc[opponent_df.index, 'rounds_loss_vs_'+opponent] = loss_his
        
        df.loc[:, 'rounds_won_vs_'+opponent].fillna(method='ffill', inplace=True)
        df.loc[:, 'rounds_loss_vs_'+opponent].fillna(method='ffill', inplace=True)
        
        df.loc[:, 'rounds_won_vs_'+opponent].fillna(0, inplace=True)
        df.loc[:, 'rounds_loss_vs_'+opponent].fillna(0, inplace=True)
    return df


def create_rounds_won_and_lost_vs_team_his(data): #applied to entire dataset
    data.sort_values('date', inplace = True)
    data = data.groupby('player_team_name').apply(count_rounds_won_vs_opponent)
    return data


def count_rounds_won_vs_opponent_on_map(df): # parsing a df of 1 team
    opponent = df.player_team_opponent.values[0]
    map_name = df.map.values[0]

    won_his = df.loc[:, 'rounds_won'].expanding(1).sum()
    loss_his = df.loc[:,'rounds_lost'].expanding(1).sum()
        
    df.loc[:, 'rounds_won_vs_'+opponent+'_on_'+map_name] = won_his
    df.loc[:, 'rounds_loss_vs_'+opponent+'_on_'+map_name] = loss_his

    return df


def create_rounds_won_and_lost_vs_team_by_map_his(data):
    data.sort_values('date', inplace = True)
    data = data.groupby(['player_team_name', 'player_team_opponent', 'map']).apply(count_rounds_won_vs_opponent_on_map)
    
    for team, opponent, map_name in itertools.product(data.player_team_name.unique(), data.player_team_opponent.unique(), data.map.unique()):
        if 'rounds_won_vs_'+opponent+'_on_'+map_name not in data.columns:
            data.loc['rounds_won_vs_'+opponent+'_on_'+map_name] = 0
            data.loc['rounds_loss_vs_'+opponent+'_on_'+map_name] = 0
            continue
        col = data.loc[data.player_team_name == team, 'rounds_won_vs_'+opponent+'_on_'+map_name].fillna(method='ffill')
        data.loc[data.player_team_name == team, 'rounds_won_vs_'+opponent+'_on_'+map_name] = col
        col = data.loc[data.player_team_name == team, 'rounds_loss_vs_'+opponent+'_on_'+map_name].fillna(method='ffill')
        data.loc[data.player_team_name == team, 'rounds_loss_vs_'+opponent+'_on_'+map_name] = col
        col = data.loc[data.player_team_name == team, 'rounds_won_vs_'+opponent+'_on_'+map_name].fillna(0)
        data.loc[data.player_team_name == team, 'rounds_won_vs_'+opponent+'_on_'+map_name] = col
        col = data.loc[data.player_team_name == team, 'rounds_loss_vs_'+opponent+'_on_'+map_name].fillna(0)
        data.loc[data.player_team_name == team, 'rounds_loss_vs_'+opponent+'_on_'+map_name] = col
    return data
    
    
def create_round_his_cols(data):
    original_col_order = data.columns.tolist()
    data = create_rounds_won_rounds_loss_columns(data)
    data = create_rounds_won_and_lost_vs_team_his(data)
    data = create_rounds_won_and_lost_vs_team_by_map_his(data)
    new_cols = data.columns[~data.columns.isin(original_col_order)].tolist()
    data = data.loc[:, original_col_order + new_cols]
    return data

In [10]:
import pandas as pd
df = pd.read_pickle('aggregated_dataset.pkl')

In [11]:
tester = create_map_win_loss_and_per_his_columns(data)
#tester = create_round_his_cols(tester)

In [12]:
tester

,match_id,player_team_name,date,team_A_name,team_B_name,series_id,map,winner_of_match,loser_of_match,player_team_opponent,index,round_num,half,team_A,team_B,team_A_score,team_B_score,match_num,+/-,ADR,Rating2.0,seang@res_who_kill_who,SicK_who_kill_who,seang@res_awp_kills,daps_first_kills,SicK_awp_kills,devoduvek_first_kills,AnJ_first_kills,AmaNEk_first_kills,ShahZaM_first_kills,AmaNEk_awp_kills,AnJ_who_kill_who,daps_who_kill_who,seang@res_first_kills,AnJ_awp_kills,CeRq_first_kills,ShahZaM_awp_kills,FugLy_who_kill_who,Brehze_first_kills,devoduvek_awp_kills,Brehze_who_kill_who,SicK_first_kills,ShahZaM_who_kill_who,devoduvek_who_kill_who,CeRq_who_kill_who,CeRq_awp_kills,Brehze_awp_kills,daps_awp_kills,FugLy_awp_kills,FugLy_first_kills,AmaNEk_who_kill_who,jks_who_kill_who,NAF_awp_kills,FalleN_first_kills,NAF_who_kill_who,fer_awp_kills,Nifty_first_kills,Nifty_who_kill_who,FalleN_who_kill_who,AZR_awp_kills,felps_awp_kills,USTILO_awp_kills,TACO_who_kill_who,AZR_who_kill_who,FalleN_awp_kills,fer_first_kills,coldzera_who_kill_who,USTILO_first_kills,coldzera_awp_kills,felps_first_kills,TACO_first_kills,AZR_first_kills,fer_who_kill_who,felps_who_kill_who,NAF_first_kills,USTILO_who_kill_who,jks_first_kills,Nifty_awp_kills,coldzera_first_kills,jks_awp_kills,TACO_awp_kills,mixwell_who_kill_who,HS_awp_kills,friberg_first_kills,allu_who_kill_who,mixwell_awp_kills,HS_first_kills,Magisk_awp_kills,Magisk_who_kill_who,Magisk_first_kills,mixwell_first_kills,allu_first_kills,allu_awp_kills,HS_who_kill_who,friberg_who_kill_who,friberg_awp_kills,FNS_awp_kills,koosta_who_kill_who,FNS_first_kills,Rickeh_first_kills,reltuC_who_kill_who,FNS_who_kill_who,nahtE_awp_kills,reltuC_first_kills,Rickeh_awp_kills,Rickeh_who_kill_who,reltuC_awp_kills,koosta_awp_kills,nahtE_first_kills,koosta_first_kills,nahtE_who_kill_who,SEMPHIS_first_kills,roca_first_kills,Drone_who_kill_who,DAVEY_who_kill_who,m1tch_awp_kills,m1tch_first_kills,SEMPHIS_awp_kills,roca_awp_kills,SEMPHIS_who_kill_who,DAVEY_first_kills,Drone_first_kills,DAVEY_awp_kills,Drone_awp_kills,m1tch_who_kill_who,roca_who_kill_who,Skadoodle_awp_kills,shroud_who_kill_who,Stewie2K_who_kill_who,Skadoodle_who_kill_who,Stewie2K_awp_kills,autimatic_first_kills,shroud_awp_kills,n0thing_who_kill_who,n0thing_first_kills,autimatic_who_kill_who,Skadoodle_first_kills,Stewie2K_first_kills,n0thing_awp_kills,autimatic_awp_kills,shroud_first_kills,bee_who_kill_who,ryx_awp_kills,Wardell_who_kill_who,CONNOR93_who_kill_who,seb_first_kills,CONNOR93_awp_kills,ryx_who_kill_who,seb_who_kill_who,CONNOR93_first_kills,...,rounds_loss_vs_Renegades_on_Train,rounds_loss_vs_Rogue,rounds_loss_vs_Rogue_on_Cache,rounds_loss_vs_Rogue_on_Cobblestone,rounds_loss_vs_Rogue_on_Inferno,rounds_loss_vs_Rogue_on_Mirage,rounds_loss_vs_Rogue_on_Nuke,rounds_loss_vs_Rogue_on_Overpass,rounds_loss_vs_Rogue_on_Train,rounds_loss_vs_SK,rounds_loss_vs_SK_on_Cache,rounds_loss_vs_SK_on_Cobblestone,rounds_loss_vs_SK_on_Inferno,rounds_loss_vs_SK_on_Mirage,rounds_loss_vs_SK_on_Overpass,rounds_loss_vs_SK_on_Train,rounds_loss_vs_Splyce,rounds_loss_vs_Splyce_on_Cache,rounds_loss_vs_Splyce_on_Cobblestone,rounds_loss_vs_Splyce_on_Inferno,rounds_loss_vs_Splyce_on_Mirage,rounds_loss_vs_Splyce_on_Nuke,rounds_loss_vs_Splyce_on_Overpass,rounds_loss_vs_Splyce_on_Train,rounds_loss_vs_compLexity,rounds_loss_vs_compLexity_on_Cache,rounds_loss_vs_compLexity_on_Cobblestone,rounds_loss_vs_compLexity_on_Inferno,rounds_loss_vs_compLexity_on_Mirage,rounds_loss_vs_compLexity_on_Nuke,rounds_loss_vs_compLexity_on_Overpass,rounds_loss_vs_compLexity_on_Train,rounds_lost,rounds_won,rounds_won_vs_CLG,rounds_won_vs_CLG_on_Cache,rounds_won_vs_CLG_on_Cobblestone,rounds_won_vs_CLG_on_Inferno,rounds_won_vs_CLG_on_Mirage,rounds_won_vs_CLG_on_Overpass,rounds_won_vs_CLG_on_Train,rounds_won_vs_Cloud9,rounds_won_vs_Cloud9_on_Cache,rounds_won_vs_Cloud9_on_Cobblestone,rounds_won_vs_Cloud9_on_Inferno,rounds_won_vs_Cloud9_on_Mirage,rounds_won_vs_Cloud9_on_Overpass,rounds_won_vs_Cloud9_on_Train,rounds_won_vs

In [3]:
df.to_excel('aggregated_dataset.xlsx')

In [4]:
df.shape

(1216, 686)

In [299]:
tester.loc[(tester.player_team_name == 'NRG')&(tester.player_team_opponent=='SK'), 'map'].unique()

array(['Mirage', 'Cache', 'Cobblestone'], dtype=object)

In [298]:
tester.loc[tester.player_team_name == 'NRG','rounds_loss_vs_SK_on_Overpass']

20      0.0
19      0.0
21      0.0
69      0.0
70      0.0
93      0.0
96      0.0
100     0.0
103     0.0
112     0.0
114     0.0
123     0.0
126     0.0
127     0.0
131     0.0
133     0.0
202     0.0
204     0.0
207     0.0
206     0.0
236     0.0
260     0.0
265     0.0
307     0.0
311     0.0
318     0.0
319     0.0
341     0.0
343     0.0
345     0.0
       ... 
826     0.0
828     0.0
830     0.0
856     0.0
859     0.0
904     0.0
906     0.0
909     0.0
912     0.0
910     0.0
913     0.0
977     0.0
980     0.0
989     0.0
992     0.0
1020    0.0
1026    0.0
1042    0.0
1047    0.0
1054    0.0
1061    0.0
1078    0.0
1113    0.0
1115    0.0
1132    0.0
1141    0.0
1144    0.0
1184    0.0
1185    0.0
1186    0.0
Name: rounds_loss_vs_SK_on_Overpass, Length: 99, dtype: float64

In [237]:
tester.loc[(tester.player_team_name == 'NRG')&(tester.player_team_opponent=='CLG')].to_excel('map_team_breakdown_sanity_check.xlsx')

In [224]:
def create_rounds_won_vs_team_his(data): # team rounds won vs another team    !6!
    grouping = data.groupby(['match_id','map','team_A_name','team_B_name', 'team_A_score', 'team_B_score'])['round_num'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    grouping = grouping.groupby(['team_A_name', 'team_B_name']).sum()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    forward = grouping.team_A_name+grouping.team_B_name
    reverse = grouping.team_B_name+grouping.team_A_name
    for idx, val in enumerate(forward):
        for idx2, val2 in enumerate(reverse):
            if val == val2 and idx < idx2:
                grouping.loc[idx,'team_A_score_Count'] += grouping.loc[idx2,'team_B_score_Count']
                grouping.loc[idx,'team_B_score_Count'] += grouping.loc[idx2,'team_A_score_Count']
                grouping.loc[idx2,'team_B_score_Count'] = grouping.loc[idx,'team_A_score_Count']
                grouping.loc[idx2,'team_A_score_Count'] = grouping.loc[idx,'team_B_score_Count']
            elif val == val2 and idx > idx2:
                grouping.loc[idx2,'team_B_score_Count'] = grouping.loc[idx,'team_A_score_Count']
                grouping.loc[idx2,'team_A_score_Count'] = grouping.loc[idx,'team_B_score_Count']
    
    grouping = grouping.drop('round_num_Count_Count', axis = 1)
    col1 = list(grouping.team_A_name.unique())
    col2  = list(grouping.team_B_name.unique())
    col = col1 + col2
    col = list(set(col))
    data = pd.merge(data,grouping, on=['team_A_name', 'team_B_name']) 
    for a in col:
        data['rd_total_his_'+ a] = 0
        data.loc[(data.player_team_name != a) & (data.team_A_name == a) , 'rd_total_his_'+ a]=data.team_B_score_Count
        data.loc[(data.player_team_name != a) & (data.team_B_name == a) , 'rd_total_his_'+ a]=data.team_A_score_Count
        bgrouping = data.groupby(['player_team_name'])['rd_total_his_'+ a].max()
        bgrouping = pd.DataFrame(bgrouping)
        bgrouping = bgrouping.reset_index()
        data = data.drop('rd_total_his_'+ a, axis = 1)
        data = pd.merge(data, bgrouping, on = 'player_team_name')
       
    return data    

In [225]:
def create_total_team_rd_map_his(data):
    grouping = data.groupby(['map','team_A_name', 'team_A_score'])['round_num'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    grouping = grouping.groupby(['map','team_A_name'])[ 'team_A_score'].sum()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    fgrouping = data.groupby(['player_team_name','map','team_B_name', 'team_B_score'])['round_num'].count()
    fgrouping = pd.DataFrame(fgrouping)
    fgrouping = fgrouping.add_suffix('_Count').reset_index()
    fgrouping = fgrouping.groupby(['map','team_B_name'])[ 'team_B_score'].sum()
    fgrouping = pd.DataFrame(fgrouping)
    fgrouping = fgrouping.add_suffix('_Count').reset_index()
    fgrouping = fgrouping.rename(index=str, columns={"team_B_name": "team_A_name", 'team_B_score_Count': 'team_A_score_Count'})
    merged = pd.concat([grouping, fgrouping], axis = 0)
    merged.groupby(['map', 'team_A_name'])['team_A_score_Count'].sum()
    merged = pd.DataFrame(merged)
    merged = merged.reset_index(drop = True)
    merged = merged.rename(index=str, columns={"team_A_name": "player_team_name", "team_A_score_Count": 'total_team_rd_map'})
    merged = merged.groupby(['player_team_name', 'map']).sum()
    merged = pd.DataFrame(merged)
    merged = merged.reset_index()
    for a in list(merged.map.unique()):
        merged.loc[:,'total_team_rd_'+ a] = 0
        merged.loc[(merged.loc[:, 'map'] == a), 'total_team_rd_'+ a] = merged.loc[:,'total_team_rd_map']
        ok_map = merged.groupby(['player_team_name'])['total_team_rd_'+ a].max()
        ok_map = pd.DataFrame(ok_map)
        ok_map = ok_map.reset_index()
        data = pd.merge(data, ok_map, on = 'player_team_name')
    return data
    

In [ ]:
#FUNCTIONS ABOVE HERE NEED TO BE WORKED ON

In [230]:
#data.to_pickle("data_converged_changed.pkl")


In [229]:
col_list = ['first_kills', 'who_kill_who','awp_kills']
data = create_fwa_dr_columns(data, col_list)
data = create_fwadr_his(data,col_list)
data = create_avdamage_his(data)
data = create_map_win_loss_his(data)
data = create_map_win_his_per(data)
data = create_rounds_won_vs_team_his(data)
data = create_total_team_rd_map_his(data)
data = create_avdamage_map_his(data)
data = create_faw_map_his(data, col_list)
data = create_matches_count(data)
data = create_opponent_team_col(data)